# Prueba LuloBank

In [1]:
import requests
from datetime import datetime
import pandas as pd
import json
from sqlalchemy import create_engine

# Extraccion API

In [2]:
BASE_URL = "https://api.tvmaze.com"
SHOWS_URL = f"{BASE_URL}/schedule/web"
DEC_DAYS = [ str(i)[:10] for i in pd.date_range(start = '2020-12-1',end='2020-12-31', freq='D')]

In [3]:
requests.get(SHOWS_URL)

<Response [200]>

In [4]:
shows = []
for day_ in DEC_DAYS:
    request = f"{SHOWS_URL}/?date={day_}"
    response = requests.get(request)
    shows.append(response.json())
    
with open('tvmaze_december_data.json', 'w') as f:
    json.dump(shows, f, indent=4)

In [5]:
shows[0][1]['_embedded']['show']['genres']

['Comedy']

# Procesamiento JSON

In [6]:
def extract_contry_table_data(json_data: dict):
    """Extrae informacion para la tabla pais
    """
    return json_data['_embedded']['show']["webChannel"]["country"]

def extract_broadcast_table_data(json_data: dict):
    """Extrae informacion para la tabla broadcast
    """
    broadcast_dict = dict(json_data)
    del broadcast_dict['_embedded']
    del broadcast_dict['rating']
    del broadcast_dict['_links']
    del broadcast_dict['image']
    broadcast_dict['show_id'] = json_data['_embedded']['show']['id']
    broadcast_dict['previousepisode'] = json_data['_embedded']['show']['_links']['previousepisode']['href']
    
    return broadcast_dict

def extract_shows_table_data(json_data: dict):
    show_dict = dict(json_data['_embedded']['show'])
    show_dict['webchannel_id'] = json_data['_embedded']['show']["webChannel"]['id']
    show_dict['dvdcountry_code'] = json_data['_embedded']['show']["webChannel"]['country']['code']
    del show_dict['schedule']
    del show_dict['webChannel']
    del show_dict['externals']
    del show_dict['_links']
    del show_dict['image']
    del show_dict['genres']
    del show_dict['rating']
    del show_dict['network']
    del show_dict['dvdCountry']
    return show_dict

def extract_webchannel_data(json_data: dict):
    webc_dict = dict(json_data['_embedded']['show']['webChannel'])
    webc_dict['country_code'] = json_data['_embedded']['show']["webChannel"]['country']['code']
    del webc_dict['country']
    return webc_dict

In [7]:
countries = []
for i, day_ in enumerate(shows):
    for j, show in enumerate(day_):
        try:
            countries.append(
                pd.DataFrame([
                    extract_contry_table_data(shows[i][j])
                ])
            )
        except:
            continue

In [8]:
broadcasts = []
for i, day_ in enumerate(shows):
    for j, show in enumerate(day_):
        try:
            broadcast_ = shows[i][j]
            broadcasts.append(
                pd.DataFrame([
                    extract_broadcast_table_data(broadcast_)
                ])
            )
        except:
            continue

In [9]:
web_channels = []
for i, day_ in enumerate(shows):
    for j, show in enumerate(day_):
        try:
            web_channel = shows[i][j]
            web_channels.append(
                pd.DataFrame([
                    extract_webchannel_data(web_channel)
                ])
            )
        except:
            continue

In [10]:
tv_shows = []
for i, day_ in enumerate(shows):
    for j, show in enumerate(day_):
        try:
            tv_show = shows[i][j]
            tv_shows.append(
                pd.DataFrame([
                    extract_shows_table_data(tv_show)
                ])
            )
        except:
            continue

# Tablas

In [11]:
tv_shows = pd.concat(tv_shows, axis=0).reset_index(drop=True).drop_duplicates()
web_channels = pd.concat(web_channels, axis=0).reset_index(drop=True).drop_duplicates()
broadcasts = pd.concat(broadcasts, axis=0).reset_index(drop=True).drop_duplicates()
countries = pd.concat(countries, axis=0).drop_duplicates().drop(0, axis=1).reset_index(drop=True).dropna()

# Carga SQLite

In [12]:
connection_sqlite = create_engine('sqlite:///tv_maze_db.db')

In [13]:
countries.to_sql('countries', connection_sqlite, if_exists='append')
web_channels.to_sql('web_channels', connection_sqlite, if_exists='append')
broadcasts.to_sql('broadcasts', connection_sqlite, if_exists='append')
tv_shows.to_sql('tv_shows', connection_sqlite, if_exists='append')

289

# Test SQL

In [14]:
pd.read_sql("SELECT * FROM countries LIMIT 5", con=connection_sqlite)

,index,name,code,timezone
0,0,"Korea, Republic of",KR,Asia/Seoul
1,1,Russian Federation,RU,Asia/Kamchatka
2,2,China,CN,Asia/Shanghai
3,3,Japan,JP,Asia/Tokyo
4,4,"Iran, Islamic Republic of",IR,Asia/Tehran


In [15]:
pd.read_sql("SELECT * FROM web_channels LIMIT 5", con=connection_sqlite)

,index,id,name,officialSite,country_code
0,0,122,V LIVE,https://www.vlive.tv/home,KR
1,1,510,Epic Media,None,RU
2,2,381,КиноПоиск HD,None,RU
3,3,51,Bilibili,None,CN
4,4,104,Tencent QQ,https://v.qq.com/,CN


In [16]:
pd.read_sql("SELECT * FROM broadcasts LIMIT 5", con=connection_sqlite)

,index,id,url,name,season,number,type,airdate,airtime,airstamp,runtime,summary,show_id,previousepisode
0,0,1979824,https://www.tvmaze.com/episodes/1979824/sim-fo...,Chanyeol's Episode 16,4,16,regular,2020-12-01,06:00,2020-11-30T21:00:00+00:00,16,<p><b>#ObtainedAConversationalSkill #WeSetUpAT...,41648,https://api.tvmaze.com/episodes/1988862
1,1,1979222,https://www.tvmaze.com/episodes/1979222/kotiki...,Серия 2,1,2,regular,2020-12-01,,2020-12-01T00:00:00+00:00,12,None,52198,https://api.tvmaze.com/episodes/1986873
2,2,2008027,https://www.tvmaze.com/episodes/2008027/lab-s-...,Лолита,2,6,regular,2020-12-01,,2020-12-01T00:00:00+00:00,29,None,52933,https://api.tvmaze.com/episodes/2245512
3,3,1964565,https://www.tvmaze.com/episodes/1964565/core-s...,Episode 9,1,9,regular,2020-12-01,10:00,2020-12-01T02:00:00+00:00,24,None,51336,https://api.tvmaze.com/episodes/1964569
4,4,2052503,https://www.tvmaze.com/episodes/2052503/wu-she...,Episode 80,1,80,regular,2020-12-01,10:00,2020-12-01T02:00:00+00:00,8,None,54033,https://api.tvmaze.com/episodes/2309427


In [17]:
pd.read_sql("SELECT * FROM tv_shows LIMIT 5", con=connection_sqlite)

,index,id,url,name,type,language,status,runtime,averageRuntime,premiered,ended,officialSite,weight,summary,updated,webchannel_id,dvdcountry_code
0,0,41648,https://www.tvmaze.com/shows/41648/sim-for-you,Sim for You,Reality,Korean,Running,16,16,2019-03-25,None,https://www.vlive.tv/video/121637,35,<p><b>Sim for You</b> is a reality series that...,1608499007,122,KR
1,1,52198,https://www.tvmaze.com/shows/52198/kotiki,Котики,Scripted,Russian,Ended,12,12,2020-11-30,2020-12-11,http://epic-media.ru/project/kotiki,16,None,1637555191,510,RU
2,2,52933,https://www.tvmaze.com/shows/52933/lab-s-anton...,LAB с Антоном Беляевым,Documentary,Russian,To Be Determined,26,25,2019-12-17,None,https://premier.one/show/lab-laboratoriya-muzy...,43,<p>Russian music artists reveal themselves fro...,1654035738,381,RU
3,3,51336,https://www.tvmaze.com/shows/51336/core-sense,Core Sense,Animation,Chinese,Running,24,24,2020-10-13,None,https://www.bilibili.com/bangumi/media/md28223064,28,"<p>The power of beginnings, the energy of the ...",1604587119,51,CN
4,4,54033,https://www.tvmaze.com/shows/54033/wu-shen-zhu...,Wu Shen Zhu Zai,Animation,Chinese,Running,8,8,2020-03-08,None,https://v.qq.com/detail/m/7q544xyrava3vxf.html,80,"<p>The protagonist Qin Chen, who was originall...",1649423444,104,CN
